# Lab 4: Constrained optimization, equality constraints

### Analise Burko & Marcos de la Torre

In [1]:
import numpy as np
import sys

We apply the Sequential Quadratic Optimization method to the problem:

$$ minimize:\   f(x_1, x_2) = e^{3x_1} + e^{−4x_2}$$

$$ subject\  to:\  h(x_1, x_2) = x_1^2 + x_2^2 − 1 = 0 $$ 

In [2]:
def func_f(x):
    '''Function to optimize'''
    x1, x2 = x
    return np.exp(3*x1) + np.exp(-4*x2)

def grad_f(x):
    '''Gradient of the function at the given point'''
    x1, x2 = x
    return np.array([3*np.exp(3*x1), -4*np.exp(-4*x2)], dtype=float)

def hessian_f(x):
    '''Hessian of the function at the given point'''
    x1, x2 = x
    return np.array([[9*np.exp(3*x1), 0], [0, 16*np.exp(-4*x2)]], dtype=float)
    
def func_h(x):
    '''Function of the equality constraint'''
    x1, x2 = x
    return x1**2 + x2**2 -1

def grad_h(x):
    '''Gradient of the equality constraint'''
    x1, x2 = x
    return np.array([2*x1, 2*x2], dtype=float)

def hessian_h(x):
    '''Hessian of the equality constraint'''
    return np.array([[2, 0], [0, 2]], dtype=float)


In [3]:
# Check that the values of step 0 are as expected
x = np.array([-1, 1], dtype=float)
lam = -1
print('Initial values: x=', x, 'lambda=', lam)
print('grad x_0:')
print(grad_f(x))
print('hessian x_0:')
print(hessian_f(x))


Initial values: x= [-1.  1.] lambda= -1
grad x_0:
[ 0.14936121 -0.07326256]
hessian x_0:
[[0.44808362 0.        ]
 [0.         0.29305022]]


### Experiment 1: Running algorithm with constant alpha=1

In [6]:
def sequential_quadratic(x_initial, lam_initial, convergence_criterion = 10e-3, print_steps=False):
    # Initial values
    x = np.array(x_initial, dtype=float)
    lam = lam_initial
    print('Initial values: x=', x, 'lambda=', lam)

    for i in range(100):  # Maximum number of iterations
        gr_f = np.expand_dims(grad_f(x), axis=1)  # Gradient of f as column vector
        gr_h = np.expand_dims(grad_h(x), axis=1)  # Gradient of h as column vector
        
        if np.linalg.norm(gr_f - lam*gr_h) < convergence_criterion:
            print('Convergence reached at step', str(i-1))
            print('Final value: x=', x, 'lambda=', lam)
            break

        # Create composite matrix A and vector b to solve the quadratic problem.
        grad_lagrange = gr_f - lam*gr_h
        hessian_lagrange = hessian_f(x) - lam*hessian_h(x)
        A = np.concatenate((np.concatenate((hessian_lagrange, -gr_h), axis=1),
                            np.concatenate((-gr_h.T, np.array([[0]])), axis=1)),
                           axis=0)
        b = np.concatenate((-grad_lagrange, np.array([[func_h(x)]], dtype=float)), axis=0)
        
        # If matrix A is singular, a different starting point is required.
        if np.linalg.cond(A) >= 1/sys.float_info.epsilon:
            print('The algorithm does not work with this starting point because the resulting matrix is '
                             'singular or ill-conditioned. Please try with a different starting point.')  
            return None
        # Obtain descent direction.
        direc = np.linalg.solve(A, b)
        # Update estimations
        x[0] = x[0] + direc[0,0]
        x[1] = x[1] + direc[1,0]
        lam = lam + direc[2,0]
        if print_steps:
            print('Step', str(i), '- descent direction', direc)
            print('Step', str(i), '- new estimates x=', x, ', lambda=', lam)
    
    return x, lam


In [5]:
sequential_quadratic(x_initial=[-1,1], lam_initial=-1)

Initial values: x= [-1.  1.] lambda= -1
Convergence reached at step 2
Final value: x= [-0.74833818  0.66332345] lambda= -0.21232390186241443


(array([-0.74833818,  0.66332345]), -0.21232390186241443)

**Comment:** Starting at (-1,1) and lambda=-1, the algorithm converges very fast into the solution.

### Experiment 2.1: Trying with other starting points

In [35]:
sequential_quadratic(x_initial=[0,0.1], lam_initial=0)

Initial values: x= [0.  0.1] lambda= 0
Convergence reached at step 9
Final value: x= [-0.74839471  0.6632797 ] lambda= -0.2122817677373336


(array([-0.74839471,  0.6632797 ]), -0.2122817677373336)

In [36]:
sequential_quadratic(x_initial=[1,1], lam_initial=0)

Initial values: x= [1. 1.] lambda= 0
Convergence reached at step 6
Final value: x= [-0.74664493  0.66790451] lambda= -0.20772098461707672


(array([-0.74664493,  0.66790451]), -0.20772098461707672)

In [37]:
sequential_quadratic(x_initial=[4,5], lam_initial=0)

Initial values: x= [4. 5.] lambda= 0
Convergence reached at step 14
Final value: x= [-0.74762403  0.664646  ] lambda= -0.21093137089499014


(array([-0.74762403,  0.664646  ]), -0.21093137089499014)

In [38]:
sequential_quadratic(x_initial=[-5,-6], lam_initial=5)

Initial values: x= [-5. -6.] lambda= 5
Convergence reached at step 24
Final value: x= [-0.74849052  0.66320119] lambda= -0.2121705340309161


(array([-0.74849052,  0.66320119]), -0.2121705340309161)

### Experiment 2.2: Testing distant points

In [7]:
sequential_quadratic(x_initial=[10,10], lam_initial=0)

Initial values: x= [10. 10.] lambda= 0
The algorithm does not work with this starting point because the resulting matrix is singular or ill-conditioned. Please try with a different starting point.


In [8]:
sequential_quadratic(x_initial=[-10,-10], lam_initial=0)

Initial values: x= [-10. -10.] lambda= 0
The algorithm does not work with this starting point because the resulting matrix is singular or ill-conditioned. Please try with a different starting point.


### **Summary of experiments:**

- With values of x of 0,0 and around, the algorithm **converges** very fast.
- With values of x of (5,5) and (-5,,-5), the algorithm still **converges** although it takes more steps.
- With values of x of (10,10) or (-10,-10), the algorithm **does not converge** anymore.

In some cases the algorithm fails because the matrix to be inverted is singular. In those cases, a small modification of the initial value usually works.

### Experiment 3: Start using merit function

#### Function to implementing backtracking gradient descent

In [25]:
def minimize_backtracking_descent(x_initial, func, grad_func, convergence_criterion=1e-3, print_steps=False):
    ''' Find a minimum of funcion "func" using backtracking gradient descent '''

    # Initial value
    x_new = x_initial
    F_new = func(x_new)

    # Iterate until end condition is met or we reach a maximum of 1000 iterations
    for num_iter in range(1000):
        x = x_new
        F = F_new
        
        # Compute gradient
        dx = grad_func(x)
        # Repeat until the gradient size is less than a defined threshold
        if (dx[0]**2 + dx[1]**2) < convergence_criterion**2:
            print('Convergence reached in {} iterations.'.format(num_iter-1))
            print('F={}, x={}'.format(F, x))
            break
        # Normalize the gradient
        dx = dx / np.linalg.norm(dx)

        alpha=1
        # Inner loop to choose an alpha value, maximum 20 iterations (alpha will be divided by 2**40, approx. 1e12)
        for j in range(20):
            x_new = x - alpha*dx
            F_new = func(x_new)
            if print_steps:
                print('num_iter={} alpha={} F={}, F_new={}, x_new={}'.format(num_iter, alpha, F, F_new, x_new))
            # Repeat until reducing alpha makes F decrease
            if F_new<F:
                break
            alpha = alpha/2
        # End of inner loop
        
    # End of outer loop
    
    return x_new

In [26]:
ro_penalty_factor=10

def func_merit(x):
    '''Merit function'''
    return func_f(x) + ro_penalty_factor*func_h(x)**2

def grad_merit(x):
    '''Gradient of the merit function at the given point'''
    return grad_f(x) + 2*ro_penalty_factor*func_h(x)*grad_h(x)


### Experiment 4: minimize merit function then apply sequential quadratic method

First we minimize the merit function. This will give us a point that is close to the optimum we look for,
where we can apply sequential quadratic optimization.

When minimizing the merit function, we may use a rather loose convergence criterion, as we do not need much precision.

In [31]:
x0 = [10,10]
x1 = minimize_backtracking_descent(x0, func_merit, grad_merit, convergence_criterion=0.1)


Convergence reached in 125 iterations.
F=0.1752542897997963, x=[-0.75541607  0.66315565]


In [32]:
sequential_quadratic(x_initial=x1, lam_initial=0)

Initial values: x= [-0.75541607  0.66315565] lambda= 0
Convergence reached at step 0
Final value: x= [-0.74703827  0.66483595] lambda= -0.2110893558350595


(array([-0.74703827,  0.66483595]), -0.2110893558350595)